## Mapping soil pH variability in the Zimbabwe Sugar Industry:

In [44]:
# Libraries
import pandas as pd
import matplotlib.pyplot as plt

In [45]:
data_2018 = pd.read_excel("data/Estates Growers & ZSAES Soil 2018.xls")
data_2019 = pd.read_excel("data/Estates Growers & ZSAES Soil 2019.xls")
data_2020 = pd.read_excel("data/Estates Growers & ZSAES Soil 2020.xls")
data_2021 = pd.read_excel("data/Estates Growers & ZSAES Soil 2021.xls")
data_2022 = pd.read_excel("data/Estates Growers & ZSAES Soil 2022.xls")
data_2023 = pd.read_excel("data/Estates Growers & ZSAES Soil 2023.xls")
data_2024 = pd.read_excel("data/Estates Growers & ZSAES Soil 2024.xls")

### starting with 2018 data

In [46]:
data_2018.head()

,Lab Ref,Source,Section,Field #,pH
0,ZS 01,C.Rusenga,HV15,1506,7.4
1,ZS 02,C.Rusenga,HV15,1506,7.46
2,ZS 03,C.Rusenga,HV15,1506,7.58
3,ZS 04,C.Rusenga,HV15,1506,7.57
4,ZS 05,C.Rusenga,HV15,1506,7.48


In [47]:
data_2018.describe()

,Lab Ref,Source,Section,Field #,pH
count,6456,6456,6146,6322,6444.00
unique,6456,265,441,3712,434.00
top,ZS6456,THZ Hippo Valley,ZSAES,M2,7.27
freq,1,1491,622,255,51.00


In [48]:
print(data_2018.isna().sum())

Lab Ref       0
Source        0
Section     310
Field #     134
pH           12
dtype: int64


In [49]:
# dropp non estate data and null data
data_2018.columns = data_2018.columns.str.strip()
data_2018_curated = data_2018.dropna(subset=['pH', 'Section'])
cats = []
for obs in data_2018['Lab Ref']:
    if(str(obs).startswith('TSS')):
        cats.append('TRI-Private')
    elif(str(obs).startswith('HSS')):
        cats.append('HVE-Private')
    elif(str(obs).startswith('MSS')):
        cats.append('MKW-Private')
    elif(str(obs).startswith('ZS')):
        cats.append('ZSAES')
    elif(str(obs).startswith('HS')):
        cats.append('HVE')
    elif(str(obs).startswith('MS')):
        cats.append('MKW')
    elif(str(obs).startswith('TS')):
        cats.append('TRI')
    else:
        cats.append('Private')
        
data_2018['zone'] = cats
data_2018.zone.value_counts()

zone
HVE            1801
TRI            1348
ZSAES          1136
Private         969
TRI-Private     446
HVE-Private     422
MKW-Private     334
Name: count, dtype: int64

In [50]:
data = data_2018[data_2018['zone'] != 'Private']

data['zone_1'] = data['zone'] + ' - ' + data['Section'].astype(str)
data.head()

C:\Users\DELL\AppData\Local\Temp\ipykernel_15088\3238703207.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['zone_1'] = data['zone'] + ' - ' + data['Section'].astype(str)


,Lab Ref,Source,Section,Field #,pH,zone,zone_1
0,ZS 01,C.Rusenga,HV15,1506,7.4,ZSAES,ZSAES - HV15
1,ZS 02,C.Rusenga,HV15,1506,7.46,ZSAES,ZSAES - HV15
2,ZS 03,C.Rusenga,HV15,1506,7.58,ZSAES,ZSAES - HV15
3,ZS 04,C.Rusenga,HV15,1506,7.57,ZSAES,ZSAES - HV15
4,ZS 05,C.Rusenga,HV15,1506,7.48,ZSAES,ZSAES - HV15


In [51]:
# Group by zone_1 and compute mean, min, and max pH
grouped_2 = data.groupby('zone_1')['pH'].agg(['mean', 'min', 'max']).reset_index()

# Find the corresponding farm/section for min and max pH in each zone_1
sect_pH_min = data.loc[data.groupby('zone_1')['pH'].idxmin(), ['zone_1', 'Field #']]
sect_pH_max = data.loc[data.groupby('zone_1')['pH'].idxmax(), ['zone_1', 'Field #']]

# Rename columns for clarity
sect_pH_min = sect_pH_min.rename(columns={'Field #': 'sect_pH_min'})
sect_pH_max = sect_pH_max.rename(columns={'Field #': 'sect_pH_max'})

# Merge back with grouped data
grouped_2 = grouped_2.merge(sect_pH_min, on='zone_1', how='left')
grouped_2 = grouped_2.merge(sect_pH_max, on='zone_1', how='left')

print(grouped_2)


TypeError: agg function failed [how->mean,dtype->object]